In [6]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
import os

In [7]:
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [10]:
llm = ChatGroq(model="gemma2-9b-it")

In [11]:
from pydantic import BaseModel, Field
class SentimentSchema(BaseModel):
    sentiment : Literal['positive','negative'] = Field(description="Sentiement of the review")


In [12]:
class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')

In [15]:

structured_model = llm.with_structured_output(SentimentSchema)
structured_model2 = llm.with_structured_output(DiagnosisSchema)

In [20]:
structured_model.invoke("What is the sentiment of the following review - The software too good")

SentimentSchema(sentiment='positive')

In [21]:
class ReviewState(TypedDict):

    review : str
    sentiment: Literal['positive','negative']
    diagnosis: dict
    response : str    

In [ ]:
def find_sentiment(state: ReviewState):

    prompt = f'For the following review find out the sentiment \n {state["review"]}'
    sentiment = structured_model.invoke(prompt)

    return {'sentiment': sentiment}

def check_sentiment(state: ReviewState)->Literal['positive_response','run_diagnosis']:

    if state['sentiment'] == 'positive' :
        return 'positive_response'
    else:
        return 'run_diagnosis'

def positive_response(state: ReviewState):
    prompt = f"""Write a warm thank-you message in response to this review:
    \n\n\"{state['review']}\"\n
Also, kindly ask the user to leave feedback on our website."""
    response = model.invoke(prompt).content
    return {'response': response}

def run_diagnosis(state:ReviewState):

    prompt = f"""Diagnose this negative review:\n\n{state['review']}\n"
    "Return issue_type, tone, and urgency.
"""